<a href="https://colab.research.google.com/github/CKeerthi/8-15-24-puzzle/blob/main/8%2C_15%2C_24_Puzzle_using_A_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

The objective of this notebook is to implement the A* algorithm with the 3 heuristics for 8, 15, and 24 puzzle:

* h1, the heuristic that is responsible for calculating Manhattan Distance
* h2, the heuristic that is responsible for calculating Misplaced Tiles
* h3, a heuristic that we were assigned to come up with, is an admissible heuristic that is the sum of h1 and h2. The weights are described later in the, 'Heuristics' section of the document


### What is the A* algorithm
The A*  algorithm is a informed search algorithm, where with the help of a heuristic(in this case one of the 3 mentioned above), it can estimate the cost of a solution. 

Examples of known informed search methods:
* best-first search algorithm
* Greedy best first-search
* RBFS(recursive best-first search)
* SMA* (simplified memory bounded A*) 

## Libraries

In [ ]:
from copy import deepcopy
import numpy as np
import time
import functools
import operator
import random
import math

## Heuristics

In [ ]:
#takes the input of current puzzle_boards and evaluates the most efficient path to goal puzzle_board
def bestsolution(board):
    bestsol = np.array([], int).reshape(-1, board_size+1)
    count = len(board) - 1
    while count != -1:
        bestsol = np.insert(bestsol, 0, board[count]['initial_state'], 0)
        count = (board[count]['parent'])
    return bestsol.reshape(-1, int(math.sqrt(board_size+1)), int(math.sqrt(board_size+1)))


# this function checks for the uniqueness of the iteration(it) board, weather it has been previously traversed or not.
def all(checkarray):
    set=[]
    for it in set:
        for checkarray in it:
            return True
        else:
            return False

# calculate Manhattan distance cost between each digit of initial_state(start board) and the goal board
# h1
def manhattan(initial_state, goal):
    a = abs(initial_state // math.sqrt(board_size+1) - goal // math.sqrt(board_size+1))
    b = abs(initial_state % math.sqrt(board_size+1) - goal % math.sqrt(board_size+1))
    mhcost = a + b
    return sum(mhcost[1:])

# will calcuates the number of misplaced tiles in the current board as compared to the goal board
# h2
def misplaced_tiles(initial_state,goal):
    mscost = np.sum(initial_state != goal) - 1
    return mscost if mscost > 0 else 0
       
# h1 and h2 are multiplied by weights, finally summed together
# h3
def weighted_astar(initial_state, goal):
    a = abs(initial_state // math.sqrt(board_size+1) - goal // math.sqrt(board_size+1))
    b = abs(initial_state % math.sqrt(board_size+1) - goal % math.sqrt(board_size+1))
    mhcost = a + b
    mscost = np.sum(initial_state != goal) - 1
    
    w_astar = 1.1 * mhcost + 2 * mscost
    return sum(w_astar)

# will indentify the coordinates of each of goal or initial board values
def coordinates(initial_state):
    pos = np.array(range(board_size + 1))
    for p, q in enumerate(initial_state):
        pos[q] = p
    return pos

# 15_initial_state
def coordinates_15(initial_state):
    pos = np.array(range(16))
    for p, q in enumerate(initial_state):
        pos[q] = p
    return pos

## Solving puzzle via A*

In [ ]:
# start of evaluation using manhattan heuristic
def evaluate(initial_state, goal):
    print(initial_state)
    steps = np.array([('up', up, -(math.sqrt(board_size+1))),('down', down, (math.sqrt(board_size+1))),('left', left, -1),('right', right,  1)],
                dtype =  [('move',  str, 1),('position', list),('head', int)])

    dtboard = [('initial_state',  list),('parent', int),('gn',  int),('hn',  int)]
    
    # initializing the parent, gn and hn, where hn is manhattan distance function call 
    costg = coordinates(goal)
    parent = -1
    gn = 0
    hn = manhattan(coordinates(initial_state), costg)
    board = np.array([(initial_state, parent, gn, hn)], dtboard)

    # We make use of priority queues with position as keys and fn as value.
    dtpriority = [('position', int),('fn', int)]
    priority = np.array( [(0, hn)], dtpriority)



    while True:
        priority = np.sort(priority, kind='mergesort', order=['fn', 'position'])     
        position, fn = priority[0]                                                 
        priority = np.delete(priority, 0, 0)  
        # sort priority queue using merge sort,the first element is picked for exploring remove from queue what we are exploring                   
        initial_state, parent, gn, hn = board[position]
        initial_state = np.array(initial_state)
        # Identify the blank square in input 
        blank = int(np.where(initial_state == 0)[0])       
        gn = gn + 1                              
        c = 1
        start_time = time.time()
        for s in steps:
            c = c + 1
            if blank not in s['position']:
                # generate new board as copy of current
                openboards = deepcopy(initial_state)                   
                openboards[blank], openboards[blank + s['head']] = openboards[blank + s['head']], openboards[blank]             
                # The all function is called, if the node has been previously explored or not
                if ~(np.all(list(board['initial_state']) == openboards, 1)).any():    
                    end_time = time.time()
                    if (( end_time - start_time ) > 2):
                        print(" The {} initial_state is unsolvable ! \n".format(board_size))
                        exit
                    # calls the manhattan function to calcuate the cost 
                    hn = manhattan(coordinates(openboards), costg)    
                    # generate and add new board in the list                    
                    q = np.array([(openboards, position, gn, hn)], dtboard)         
                    board = np.append(board, q, 0)
                    # f(n) is the sum of cost to reach node and the cost to rech fromt he node to the goal board
                    fn = gn + hn                                        
            
                    q = np.array([(len(board) - 1, fn)], dtpriority)    
                    priority = np.append(priority, q, 0)
                      # Checking if the node in openboards are matching the goal board.  
                    if np.array_equal(openboards, goal):                              
                        print(" The {} initial state is solvable ! \n".format(board_size))
                        return board, len(priority)
      
                        
    return board, len(priority)


# misplaced tiles heuristic
def evaluate_misplaced(initial_state, goal):
    print(initial_state)
    steps = np.array([('up', up, -(math.sqrt(board_size+1))),('down', down, (math.sqrt(board_size+1))),('left', left, -1),('right', right,  1)],
                dtype =  [('move',  str, 1),('position', list),('head', int)])

    dtboard = [('initial_state',  list),('parent', int),('gn',  int),('hn',  int)]

    costg = coordinates(goal)
    # initializing the parent, gn and hn, where hn is misplaced_tiles  function call  
    parent = -1
    gn = 0
    hn = misplaced_tiles(coordinates(initial_state), costg)
    board = np.array([(initial_state, parent, gn, hn)], dtboard)

    # We make use of priority queues with position as keys and fn as value.
    dtpriority = [('position', int),('fn', int)]

    priority = np.array([(0, hn)], dtpriority)
    
    while True:
        priority = np.sort(priority, kind='mergesort', order=['fn', 'position'])      
        position, fn = priority[0]       
        # sort priority queue using merge sort,the first element is picked for exploring.                                          
        priority = np.delete(priority, 0, 0)                         
        initial_state, parent, gn, hn = board[position]
        initial_state = np.array(initial_state)
        # Identify the blank square in input 
        blank = int(np.where(initial_state == 0)[0])   
        # Increase cost g(n) by 1  
        gn = gn + 1                             
        c = 1
        start_time = time.time()
        for s in steps:
            c = c + 1
            if blank not in s['position']:
                # generate new board as copy of current
                openboards = deepcopy(initial_state)         
                openboards[blank], openboards[blank + s['head']] = openboards[blank + s['head']], openboards[blank]
                # The check function is called, if the node has been previously explored or not. 
                if ~(np.all(list(board['initial_state']) == openboards, 1)).any():          
                    end_time = time.time()
                    if (( end_time - start_time ) > 2):
                        print(" The", board_size, "initial_state is unsolvable \n")
                        break
                    # calls the Misplaced_tiles function to calcuate the cost 
                    hn = misplaced_tiles(coordinates(openboards), costg) 
                    # generate and add new board in the list                    
                    q = np.array([(openboards, position, gn, hn)], dtboard)         
                    board = np.append(board, q, 0)
                    # f(n) is the sum of cost to reach node and the cost to rech fromt he node to the goal board
                    fn = gn + hn                                        
                    
                    q = np.array([(len(board) - 1, fn)], dtpriority)
                    priority = np.append(priority, q, 0)
                    # Checking if the node in openboards are matching the goal board.
                    if np.array_equal(openboards, goal):                      
                        print(" The {} initial state is solvable ! \n".format(board_size))
                        return board, len(priority)
                          
    return board, len(priority)
  
# weighted a star heuristic
def evaluate_weighted(initial_state, goal):
    print(initial_state)
    steps = np.array([('up', up, -(math.sqrt(board_size+1))),('down', down, (math.sqrt(board_size+1))),('left', left, -1),('right', right,  1)],
                dtype =  [('move',  str, 1),('position', list),('head', int)])

    dtboard = [('initial_state',  list),('parent', int),('gn',  int),('hn',  int)]

    costg = coordinates(goal)
    # initializing the parent, gn and hn, where hn is misplaced_tiles  function call  
    parent = -1
    gn = 0
    hn = weighted_astar(coordinates(initial_state), costg)
    board = np.array([(initial_state, parent, gn, hn)], dtboard)

    # We make use of priority queues with position as keys and fn as value.
    dtpriority = [('position', int),('fn', int)]

    priority = np.array([(0, hn)], dtpriority)
    
    while True:
        priority = np.sort(priority, kind='mergesort', order=['fn', 'position'])      
        position, fn = priority[0]       
        # sort priority queue using merge sort,the first element is picked for exploring.                                          
        priority = np.delete(priority, 0, 0)                         
        initial_state, parent, gn, hn = board[position]
        initial_state = np.array(initial_state)
        # Identify the blank square in input 
        blank = int(np.where(initial_state == 0)[0])   
        # Increase cost g(n) by 1  
        gn = gn + 1                             
        c = 1
        start_time = time.time()
        for s in steps:
            c = c + 1
            if blank not in s['position']:
                # generate new board as copy of current
                openboards = deepcopy(initial_state)         
                openboards[blank], openboards[blank + s['head']] = openboards[blank + s['head']], openboards[blank]
                # The check function is called, if the node has been previously explored or not. 
                if ~(np.all(list(board['initial_state']) == openboards, 1)).any():          
                    end_time = time.time()
                    if (( end_time - start_time ) > 2):
                        print(" The", board_size, "initial_state is unsolvable \n")
                        break
                    # calls the Misplaced_tiles function to calcuate the cost 
                    hn = misplaced_tiles(coordinates(openboards), costg) 
                    # generate and add new board in the list                    
                    q = np.array([(openboards, position, gn, hn)], dtboard)         
                    board = np.append(board, q, 0)
                    # f(n) is the sum of cost to reach node and the cost to rech fromt he node to the goal board
                    fn = gn + hn                                        
                    
                    q = np.array([(len(board) - 1, fn)], dtpriority)
                    priority = np.append(priority, q, 0)
                    # Checking if the node in openboards are matching the goal board.
                    if np.array_equal(openboards, goal):                      
                        print(" The {} initial state is solvable ! \n".format(board_size))
                        return board, len(priority)
                          
    return board, len(priority)


## Generating random States (Freezing)

* A consideration that had to be accounted for was the time it took to produce tangible results for 15 puzzle. To generate a solution for a completely random state was infeasible for the code that we had.
* A solution was to get tangible results in a reasonable amount of time is called **freezing**.
* The logic behind freezing is that it is easier to get to the initial state to a goal state if part of the initial state is already solved. In practice, this has made the code runnable.
* In the example of the 15 puzzle, we will keep the first 9 elements of the state constant as well as the 0, and then randomize the values of the remaining elements to produce states that are still solvable, but require less time to process.
* The function that deals with freezing is called “generate_states_15_puzzle”

In [ ]:
#Creates a lit to store all 100 random instances of either 8,15, puzzle 
master_board_list = []


  
def convert_to_2d_list(board, n):
    return [board[i:i+n] for i in range(0, len(board), n)]


def generate_states_8_puzzle():
    #board_list_frozen = [1,2,3]
    #board_list_zero = [0]
    board_list_8_unfrozen = random.sample(range(0,9), 9)
    #basically adding up the lists
    #board_list_8_puzzle = board_list_frozen + board_list_8_unfrozen + board_list_zero
    board_list_8_puzzle = board_list_8_unfrozen

    return board_list_8_puzzle

def generate_states_15_puzzle():
    board_list_frozen = [1,2,3,4,5,6,7,8,9]
    board_list_zero = [0]
    board_list_15_frozen = random.sample(range(10,16), 6)
    board_list_15_puzzle = board_list_frozen + board_list_15_frozen + board_list_zero
    return board_list_15_puzzle

def generate_states_24_puzzle():
    #note this code below is unfrozen, it could take some time to execute
    board_list_24_puzzle = random.sample(range(0,25), 25)
    return board_list_24_puzzle
  
  
def check_if_solvable(board):
    # Needed because we need to generate 100 REACHABLE states
    # It is not possible to solve an instance of 8 puzzle if 
    # number of inversions is odd in the input state
    # An inversion is when a tile with a greater number on it 
    # precedes a tile with a smaller number
    # Returns true if inversions are even
    board_flat = functools.reduce(operator.iconcat, board, [])
    count = 0

    for i in range(len(board_flat) - 1):
        for j in range(i+1, len(board_flat)):
            if board_flat[j] and board_flat[i] and board_flat[i] > board_flat[j]:
                count += 1

    return count % 2 == 0
 



## Generating Master Lists

In [ ]:
#function generates 100 8-puzzles 
def generate_8_puzzle_master_list():
    broken = False
    while len(master_board_list) != 100:
        board = generate_states_8_puzzle()
        board_2d = convert_to_2d_list(board, 3)
        true_or_false = check_if_solvable(board_2d)
        if true_or_false == True:
          board_flat = functools.reduce(operator.iconcat, board_2d, [])
          master_board_list.append(board_flat)
            

def generate_15_puzzle_master_list():
    broken = False
    while len(master_board_list) != 100:
        board = generate_states_15_puzzle()
        board_2d = convert_to_2d_list(board, 4)
        true_or_false = check_if_solvable(board_2d)
        if true_or_false == True:
          board_flat = functools.reduce(operator.iconcat, board_2d, [])
          master_board_list.append(board_flat)

def generate_24_puzzle_master_list():
    broken = False
    while len(master_board_list) != 100:
        board = generate_states_24_puzzle()
        board_2d = convert_to_2d_list(board, 5)
        true_or_false = check_if_solvable(board_2d)
        if true_or_false == True:
          board_flat = functools.reduce(operator.iconcat, board_2d, [])
          master_board_list.append(board_flat)

## Declaring Total Variables & Testing

In [ ]:
board_size = int(input("Size of board (8, 15, 24): "))
n = int(input("1. Manhattan distance \n2. Misplaced tiles  \n3. Weighted A* \n"))


# Loop through puzzle sizing 8,16,24
initial_state = []
goal = []
board_size_list = [8,15,24]
heuristics = [1,2,3] #1: Manhattan 2:Misplaced 3:Weighted A*

#for board_size in board_size_list:
if(board_size == 8):
    up = [0,1,2]
    down = [6,7,8]
    left = [0,3,6]
    right = [2,5,8]
    print("Generating 100 Random States ")
    generate_8_puzzle_master_list()
    print(len(master_board_list))
    print("")


    # Goal board       
    goal = [1,2,3,4,5,6,7,8,0]
    print("")

if(board_size == 15):
    up = [0,1,2,3]
    down = [12,13,14,15]
    left = [0,4,8, 12]
    right = [3,7,11,15]
    print("Generating 100 Random States ")
    generate_15_puzzle_master_list()
    print(len(master_board_list))
    print("")

    # User input of goal board       
    goal = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0]
    print("")


if(board_size == 24):
    up = [0,1,2,3,4]
    down = [20,21,22,23,24]
    left = [0,5,10,15]
    right = [4,9,14,19,24]
    print("Generating 100 Random States ")
    generate_24_puzzle_master_list()
    print("")

    # User input of goal board       
    goal = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,0]
    print("")


if(n == 1): 
    board_number = 0
    if goal in master_board_list:
      print("Remove happened")
      master_board_list[:] = [x for x in master_board_list if x != goal]
      print(master_board_list)
      random_state_list = master_board_list
    else:
      random_state_list = master_board_list
    print("Before Evaluate")

    for initial_state in random_state_list:
      board, visited = evaluate(initial_state, goal)
      print("After evaluate")
      bestpath = bestsolution(board)
      print(str(bestpath).replace('[', ' ').replace(']', ''))
      totalmoves = len(bestpath) - 1
      print('Steps to reach goal:',totalmoves)
      visit = len(board) - visited
      board_number = board_number + 1
      if board_size == 8:
        total_moves_manhattan_8 += totalmoves
        total_nodes_visited_manhattan_8 += visit
        total_generated_manhattan_8 += len(board)
      elif board_size == 15:
        total_moves_manhattan_15 += totalmoves
        total_nodes_visited_manhattan_15 += visit
        total_generated_manhattan_15 += len(board)
      elif board_size == 24:
        total_moves_manhattan_24 += totalmoves
        total_nodes_visited_manhattan_24 += visit
        total_generated_manhattan_24 += len(board)
      print('Total nodes visited: ',visit, "\n")
      print('Total generated:', len(board))
      print("Board Number:", board_number)

if(n == 2):
    board_number = 0
    if goal in master_board_list:
      print("Remove happened")
      master_board_list[:] = [x for x in master_board_list if x != goal]
      print(master_board_list)
      random_state_list = master_board_list
    else:
      random_state_list = master_board_list
    print("Before Evaluate")
    total_moves_15 = 0
    total_nodes_visited_15 = 0
    total_generated_15 = 0
    for initial_state in random_state_list:
      board, visited = evaluate(initial_state, goal)
      print("After evaluate")
      bestpath = bestsolution(board)
      print(str(bestpath).replace('[', ' ').replace(']', ''))
      totalmoves = len(bestpath) - 1
      print('Steps to reach goal:',totalmoves)
      visit = len(board) - visited
      board_number = board_number + 1
      if board_size == 8:
        total_moves_misplaced_8 += totalmoves
        total_nodes_visited_misplaced_8 += visit
        total_generated_misplaced_8 += len(board)
      elif board_size == 15:
        total_moves_misplaced_15 += totalmoves
        total_nodes_visited_misplaced_15 += visit
        total_generated_misplaced_15 += len(board)
      elif board_size == 24:
        total_moves_misplaced_24 += totalmoves
        total_nodes_visited_misplaced_24 += visit
        total_generated_misplaced_24 += len(board)
      print('Total nodes visited: ',visit, "\n")
      print('Total generated:', len(board))
      print("Board Number:", board_number)

if(n == 3):
    board_number = 0
    if goal in master_board_list:
      print("Remove happened")
      master_board_list[:] = [x for x in master_board_list if x != goal]
      print(master_board_list)
      random_state_list = master_board_list
    else:
      random_state_list = master_board_list
    print("Before Evaluate")
    total_moves_24 = 0
    total_nodes_visited_24 = 0
    total_generated_24 = 0
    for initial_state in random_state_list:
      board, visited = evaluate(initial_state, goal) 
      print("After evaluate")
      bestpath = bestsolution(board)
      print(str(bestpath).replace('[', ' ').replace(']', ''))
      totalmoves = len(bestpath) - 1
      print('Steps to reach goal:',totalmoves)
      visit = len(board) - visited
      board_number = board_number + 1
      if board_size == 8:
        total_moves_weighted_8 += totalmoves
        total_nodes_visited_weighted_8 += visit
        total_generated_weighted_8 += len(board)
      elif board_size == 15:
        total_moves_weighted_15 += totalmoves
        total_nodes_visited_weighted_15 += visit
        total_generated_weighted_15 += len(board)
      elif board_size == 24:
        total_moves_weighted_24 += totalmoves
        total_nodes_visited_weighted_24 += visit
        total_generated_weighted_24 += len(board)
      print('Total nodes visited: ',visit, "\n")
      print('Total generated:', len(board))
      print("Board Number:", board_number)


Size of board (8, 15, 24): 8
1. Manhattan distance 
2. Misplaced tiles  
3. Weighted A* 
1
Generating 100 Random States 
100


Before Evaluate
[8, 2, 6, 5, 7, 4, 1, 0, 3]
 The 8 initial state is solvable ! 

After evaluate
   8 2 6
   5 7 4
   1 0 3

   8 2 6
   5 0 4
   1 7 3

   8 2 6
   0 5 4
   1 7 3

   8 2 6
   1 5 4
   0 7 3

   8 2 6
   1 5 4
   7 0 3

   8 2 6
   1 0 4
   7 5 3

   8 2 6
   1 4 0
   7 5 3

   8 2 0
   1 4 6
   7 5 3

   8 0 2
   1 4 6
   7 5 3

   0 8 2
   1 4 6
   7 5 3

   1 8 2
   0 4 6
   7 5 3

   1 8 2
   4 0 6
   7 5 3

   1 8 2
   4 6 0
   7 5 3

   1 8 2
   4 6 3
   7 5 0

   1 8 2
   4 6 3
   7 0 5

   1 8 2
   4 0 3
   7 6 5

   1 0 2
   4 8 3
   7 6 5

   1 2 0
   4 8 3
   7 6 5

   1 2 3
   4 8 0
   7 6 5

   1 2 3
   4 8 5
   7 6 0

   1 2 3
   4 8 5
   7 0 6

   1 2 3
   4 0 5
   7 8 6

   1 2 3
   4 5 0
   7 8 6

   1 2 3
   4 5 6
   7 8 0
Steps to reach goal: 23
Total nodes visited:  1364 

Total generated: 2130
Board Number: 1
[0, 4, 6, 1, 8,

KeyboardInterrupt: ignored